In [131]:
import numpy as np
import pandas as pd
import hplib
numbers = {'Tin': [1,2,3,4,5,6,7,8,9,10]}
Tout=[10,20,30,40,50,60,70,80,90,100]
df = pd.DataFrame(numbers)
df['Tout']=Tout
df.loc[df['Tin']>5,'Tin']=5
df.Tout=df.Tout
df

Group=parameters['Group'].array[0]
k1=parameters['p1_P_th [1/°C]'].array[0]
k2=parameters['p2_P_th [1/°C]'].array[0]
k3=parameters['p3_P_th [-]'].array[0]
k4=parameters['p1_P_el [1/°C]'].array[0]
k5=parameters['p2_P_el [1/°C]'].array[0]
k6=parameters['p3_P_el [-]'].array[0]
k7=parameters['p1_COP [-]'].array[0]
k8=parameters['p2_COP [-]'].array[0]
k9=parameters['p3_COP [-]'].array[0]
Pel_ref=parameters['P_el_ref [W]'].array[0]
P_th_ref=parameters['P_th_ref [W]'].array[0]
df['COP']=k7*df.Tin+k8*df.Tout+k9
df['Pel']=(k4*df.Tin+k5*df.Tout+k6)*Pel_ref
df

,Tin,Tout,COP,Pel
0,1,10,6.336323,1815.431345
1,2,20,5.609181,2268.011678
2,3,30,4.882038,2720.592012
3,4,40,4.154896,3173.172346
4,5,50,3.427753,3625.752679
5,5,60,2.546066,4299.767173
6,5,70,1.664380,4973.781667
7,5,80,0.782693,5647.796160
8,5,90,-0.098994,6321.810654
9,5,100,-0.980681,6995.825147


In [136]:
def simulateSeries(T_in_primary, T_in_secondary, parameters):
    #inputs  
    # T_amb [°C], ambient temperature, neccesary for inverter heat pumps
    # T_in_primary [°C] as Pandas.core.series.Series, source temperature (air or ground)
    # T_in_secondary [°C]as Pandas.core.series.Series
    # parameters -> dataframe from function getParameters()


    df = T_in_primary.to_frame()
    delta_T=5 # Inlet temperature is supposed to be heated up by 5 K
    df['Tout']=T_in_secondary+delta_T 
    Model=parameters['Model'].array[0]
    Group=parameters['Group'].array[0]
    k1=parameters['p1_P_th [1/°C]'].array[0]
    k2=parameters['p2_P_th [1/°C]'].array[0]
    k3=parameters['p3_P_th [-]'].array[0]
    k4=parameters['p1_P_el [1/°C]'].array[0]
    k5=parameters['p2_P_el [1/°C]'].array[0]
    k6=parameters['p3_P_el [-]'].array[0]
    k7=parameters['p1_COP [-]'].array[0]
    k8=parameters['p2_COP [-]'].array[0]
    k9=parameters['p3_COP [-]'].array[0]
    Pel_ref=parameters['P_el_ref [W]'].array[0]
    P_th_ref=parameters['P_th_ref [W]'].array[0]
    if Model=='Generic':
        x=parameters['T_primary'].array[0]
        y=parameters['T_secondary'].array[0]
        P_el_para=k4*x+k5*y+k6
        Pel_ref=Pel_ref/P_el_para
    
    if Group==1 or Group==2 or Group==3:
        df['COP']=k7*df.Tin+k8*df.Tout+k9
        df.loc[df['Tin']>5,'Tin']=5#minimum electrical Power at 5°C
        df['Pel']=(k4*df.Tin+k5*df.Tout+k6)*Pel_ref
        df['P_th']=df['Pel']*df['COP']
        df.loc[df['COP']<1,'Pel']=P_th_ref
        df.loc[df['COP']<1,'P_th']=P_th_ref
        df.loc[df['COP']<1,'COP']=1
    elif Group==4 or Group==5 or Group==6:
        df['Pel']=(k4*df.Tin+k5*df.Tout+k6)*Pel_ref
        df['COP']=k7*df.Tin+k8*df.Tout+k9
        df['P_th']=df['Pel']*df['COP']
        df.loc[df['COP']<1,'Pel']=P_th_ref
        df.loc[df['COP']<1,'P_th']=P_th_ref
        df.loc[df['COP']<1,'COP']=1
    else:
        df['Pel']==0
        df['COP']=0
        df['P_th']=0
        df.loc[df['COP']<1,'Pel']=P_th_ref
        df.loc[df['COP']<1,'P_th']=P_th_ref
        df.loc[df['COP']<1,'COP']=1

    # massflow
    cp = 4200 # J/(kg*K), specific heat capacity of water
    df['m_dot'] = df['P_th'] / (delta_T * cp)
    return df

In [137]:
df1=simulateSeries(df['Tin'],df['Tout'],parameters)
df1

,Tin,Tout,COP,Pel,Pth,m_dot
0,1,15,5.895480,2400.966652,14154.850986,0.674041
1,2,25,5.168337,2905.803488,15018.173061,0.715151
2,3,35,4.441195,3410.640324,15147.318426,0.721301
3,4,45,3.714052,3915.477160,14542.287081,0.692490
4,5,55,2.986910,4420.313997,13203.079026,0.628718
5,5,65,2.105223,5172.152552,10888.534496,0.518502
6,5,75,1.223536,5923.991108,7248.217759,0.345153
7,5,85,1.000000,10000.000000,10000.000000,0.476190
8,5,95,1.000000,10000.000000,10000.000000,0.476190
9,5,105,1.000000,10000.000000,10000.000000,0.476190


In [134]:
parameters

,Model,P_th_ref [W],P_el_ref [W],COP_ref,Group,p1_P_th [1/°C],p2_P_th [1/°C],p3_P_th [-],p1_P_el [1/°C],p2_P_el [1/°C],p3_P_el [-],p1_COP [-],p2_COP [-],p3_COP [-],T_primary,T_secondary
0,Generic,10000.0,7158.783065,1.396885,1.0,-0.030227,-0.003634,0.91786,-0.030932,0.009415,0.190375,0.154544,-0.088169,7.063466,-7.0,52.0
